In [1]:
%matplotlib 
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt4Agg


In [2]:
import sys
sys.path.append('/mnt/Data/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')

In [76]:
import matplotlib as mpl
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import scipy.signal as scsig
import spectrum as sp
import signale
import cPickle as pkl
import time
import statsmodels.api as sm
import colormaps as cm
import seaborn as sns
import signale.tools as tools
import os
import itertools as it
from scipy import stats
from scipy.signal import gaussian

In [4]:
def MA(array,ord=2):
    ma = np.array([array[ii-ord/2:ii+ord/2].mean() for ii in range(array.size) if ii >= ord ],dtype=float)
    ma = np.insert(ma,0,array[0:ord/2])
    ma = np.append(ma,array[-(ord/2)-1:-1])
    return ma
def zScore(array):
    return(array - np.average(array))/ np.std(array)

In [5]:
localPath = '/mnt/Data/ephysdata/Gerbils/'

In [6]:
lfpPaths = []
for lfp in tools.locate('*.lfp',localPath):
    lfpPaths.append(os.path.join(lfp[0],lfp[1]))
lfpPaths = sorted(lfpPaths)
#lfpPaths = [ path for path in lfpPaths if (path.find('leep')>0 and path.find('CSC4')>0) ]
lfpPaths

['/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-04_sleep1_csc4.lfp',
 '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-04_sleep2_csc4.lfp',
 '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-04_sleep3_csc4.lfp',
 '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-05_sleep1_csc4.lfp',
 '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-05_sleep2_csc4.lfp',
 '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-05_sleep3_csc4.lfp',
 '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-07_sleep1_csc4.lfp',
 '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-07_sleep2_csc4.lfp',
 '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-07_sleep3_csc4.lfp',
 '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-09_sleep1_csc4.lfp',
 '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-09_sleep2_csc4.lfp',
 '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-09_sleep3_csc4.lfp',
 '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-10_sleep1_csc4.lfp',
 '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-10_sleep2_csc4.lfp',
 '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-10_sleep3_csc4.l

In [177]:
SleepEpochs = pd.read_pickle('/mnt/Data/ephysdata/Gerbils/SleepEpochs.pd')
df = pd.read_pickle('/mnt/Data/ephysdata/Gerbils/psdData.pd')

In [80]:
gaussian(100,11).sum()

27.572760993510304

In [178]:
sns.tsplot(value='psd',time='f',condition='epoch',unit='epochNo',data=df)

In [15]:
for idx,row in SleepEpochs.iterrows():
    sess = row.session.split('0')[0]+row.session.split('0')[1]
    for pth in lfpPaths:
        if pth.find(row.date)>0 and pth.find(sess)>0:
            lfp = pkl.load(open(pth,'rb'))

In [174]:
SleepEpochs# = SleepEpochs[SleepEpochs.animal=='G6']
aaa = np.unique(SleepEpochs.session)
bbb = np.unique(SleepEpochs.date)
dBdf = pd.DataFrame([],columns=['epochIdx','dB','f','epoch','session','date','animal'])
kkk = 0
for date,session in it.product(bbb,aaa):
    sess = session.split('0')[0] + session.split('0')[1]
    print sess,date
    for pth in lfpPaths:
        if pth.find(date)>0 and pth.find(sess)>0:
            lfp = pkl.load(open(pth,'rb'))
    t_init = lfp.timeAxis[0]
    df = SleepEpochs[(SleepEpochs.session==session) & (SleepEpochs.date==date)]
    for idx,row in df.iterrows():
        lfpChunk = lfp.signal_white[np.logical_and(lfp.timeAxis>t_init+1e3*row.t0,lfp.timeAxis<t_init+1e3*row.t1)]
        p = sp.Periodogram(lfpChunk,NFFT=2**14,sampling=2e3)
        p();
        f = np.array(p.frequencies())
        sPSD = np.convolve(p.psd*np.power(f,0),gaussian(20,5),'same')
        db = 10*np.log10(sPSD/sPSD[0])
        epochIdx = np.ones((f<500).sum())
        epochIdx = np.int16(epochIdx)
        if kkk:
            epochIdx = np.ones((f<500).sum())
            epochIdx = epochIdx*(kkk+1)
            epochIdx = np.int16(epochIdx)
        dataArr =  np.array([epochIdx,
                             db[f<500].T,
                             f[f<500].T,
                             np.array([row.epoch]*(f<500).sum()).T,
                             np.array([row.session]*(f<500).sum()).T,
                             np.array([row.date]*(f<500).sum()).T,
                             np.array([row.animal]*(f<500).sum()).T])
        kkk+=1
        dBdf = dBdf.append(pd.DataFrame(dataArr.T,columns=dBdf.columns),ignore_index=True)
        

sleep1 2011-08-05
sleep2 2011-08-05
sleep3 2011-08-05
sleep1 2011-08-07
sleep2 2011-08-07
sleep3 2011-08-07
sleep1 2011-08-09
sleep2 2011-08-09
sleep3 2011-08-09
sleep1 2011-08-10
sleep2 2011-08-10
sleep3 2011-08-10
sleep1 2011-09-19
sleep2 2011-09-19
sleep3 2011-09-19
sleep1 2011-09-23
sleep2 2011-09-23
sleep3 2011-09-23


In [175]:
cols = ['epochIdx','dB','f']
dBdf.ix[:, cols] = dBdf.ix[:, cols].apply(pd.to_numeric)

In [176]:
sns.tsplot(value='dB',time='f',condition='epoch',unit='epochIdx',data=dBdf)

In [149]:
pl.scatter(f[100:2000],db[100:2000])

In [145]:
#slope, intercept, r_value, p_value, std_err = 
stats.linregress(f[1000:2000],db[1000:2000])

LinregressResult(slope=-0.057170052706652229, intercept=-8.0581188967101181, rvalue=-0.69365158033537988, pvalue=2.319935137974389e-144, stderr=0.0018792380614175938)

In [140]:
pl.plot(f,db)

In [144]:
f[2000]

244.140625

In [121]:
f

array([  0.00000000e+00,   1.22070312e-01,   2.44140625e-01, ...,
         9.99755859e+02,   9.99877930e+02,   1.00000000e+03])